# Predicción de tópicos
Se desea extraer los tópicos principales de cada semana dentro del ámbito político

## Librerias

In [31]:
import pandas as pd
import re # expresiones regulares (regex)
from nltk.corpus import stopwords

# Preprocesamiento

In [22]:
# Archivo npr.csv es para hacer pruebas

df = pd.read_csv('noticias-economicas-sample.csv')

textoPrueba = df[['text']]

print(textoPrueba.shape)
textoPrueba.head(10)

(10000, 1)


,text
0,Angela Merkel ayer en Berlin tras la cumbre te...
1,\n\n\n\n\nEnrique Quemada Clariana\nFusiones y...
2,¿Cuántas veces no ha querido ser su propio jef...
3,"El ministro de Sanidad, Salvador Illa, momento..."
4,El Pleno del Congreso ha aprobado el proyecto ...
5,Vacunarse de la gripe para aliviar los hospita...
6,"Elon Musk, fundador de Tesla. EFE\n\nLa compañ..."
7,Una usuaria consulta la aplicación de banca ar...
8,Las consecuencias socioeconómicas de la pandem...
9,La aerolínea Ryanair anunció este viernes que ...


### Normalizamos el texto, quitamos mayúsculas y signos de puntuación

In [12]:
# FUNCIÓN PARA NORMALIZAR EL TEXTO
def noPuntuacion(text): return re.findall(r'[a-zA-ZñÑáéíóúÁÉÍÓÚüÜ]+', text.lower())
# Elimina signos de puntuación y solo deja letras en minúsculas. No elimina tildes.

In [30]:
textoPruebaNorm = pd.DataFrame(textoPrueba["text"].apply(noPuntuacion).apply(lambda x: " ".join(x)))
textoPruebaNorm.columns = ["texto normalizado"]
textoPruebaNorm.head(10)

,texto normalizado
0,angela merkel ayer en berlin tras la cumbre te...
1,enrique quemada clariana fusiones y adquisicio...
2,cuántas veces no ha querido ser su propio jefe...
3,el ministro de sanidad salvador illa momentos ...
4,el pleno del congreso ha aprobado el proyecto ...
5,vacunarse de la gripe para aliviar los hospita...
6,elon musk fundador de tesla efe la compañía in...
7,una usuaria consulta la aplicación de banca ar...
8,las consecuencias socioeconómicas de la pandem...
9,la aerolínea ryanair anunció este viernes que ...
